In [28]:
import pandas as pd

In [29]:
all_data = pd.read_excel('DHC_Analysis.xlsx', sheet_name='Data')
LH_customers = pd.read_csv('Data/LH2_contacts.csv')
MGO_customers = pd.read_csv('Data/MGO_contacts.csv')

In [30]:
LH_companies = list(LH_customers['Compnay'].unique())
MGO_companies = list(MGO_customers['Company'].str.lower().unique())

In [31]:
all_data = all_data.rename(columns = {'Isell Login':'Isell_Login',
                           'Advantage Customer #': 'Advantage_Customer',
                           'Company Name':'Company_Name',
                           'Customer Category':'Customer_Category',
                           'Full Profiles':'Full_Profiles',
                           'Full Rev':'Full_Rev',
                           'Reapp Profiles':'Reapp_Profiles',
                           ' Reapp Rev':'Reapp_Rev',
                           'PA Profiles':'PA_Profiles',
                           'PA Rev':'PA_Rev',
                           'Total Profiles':'Total_Profiles',
                           'Total Rev':'Total_Rev',
                           'CM Rev':'CM_Rev',
                           'Customer Number':'Customer_Number'
                          })

In [50]:
all_data.dtypes

Isell_Login            object
Advantage_Customer      int64
Company_Name           object
Customer_Category      object
Full_Profiles           int64
Full_Rev                int64
Reapp_Profiles          int64
Reapp_Rev               int64
PA_Profiles             int64
PA_Rev                  int64
Total_Profiles          int64
Total_Rev               int64
CM_Rev                  int64
YEAR                    int64
Customer_Number         int64
MonthName              object
x                     float64
dtype: object

### 1.  How many of the contacts began using reappointments and PA profiles?  

In [20]:
reapp_pa_dict ={}
for num in LH_companies:
    reapp_pa_dict[num]={}
    reapp_pa_dict[num]['Reappointments_Before_Campaign']=0
    reapp_pa_dict[num]['PA_Profiles_Before_Campaign']=0
    reapp_pa_dict[num]['Reappointments_After_Campaign']=0
    reapp_pa_dict[num]['PA_Profiles_After_Campaign']=0
    reapp_pa_dict[num]['Reappointments_2019']=0
    reapp_pa_dict[num]['Reappointments_2018']=0
    reapp_pa_dict[num]['PA_Profiles_2019']=0
    reapp_pa_dict[num]['PA_Profiles_2018']=0
    reapp_pa_dict[num]['Reappointments_Revenue_2019']=0
    reapp_pa_dict[num]['Reappointments_Revenue_2018']=0
    reapp_pa_dict[num]['PA_Profiles_Revenue_2019']=0
    reapp_pa_dict[num]['PA_Profiles_Revenue_2018']=0
    reapp_pa_dict[num]['Reappointment_Revenue_During_Campaign']=0
    reapp_pa_dict[num]['PA_Profile_Revenue_During_Campaign']=0

In [21]:
for row in all_data.itertuples():
    num = str(row.Company_Name).lower()
    if num in reapp_pa_dict.keys():
        if row.YEAR < 2019:
            reapp_pa_dict[num]['Reappointments_Before_Campaign']+=row.Reapp_Profiles
            reapp_pa_dict[num]['PA_Profiles_Before_Campaign']+=row.PA_Profiles
        elif row.YEAR == 2019:
            if row.MonthName in ['September','October','November','December']:
                reapp_pa_dict[num]['Reappointments_After_Campaign']+=row.Reapp_Profiles
                reapp_pa_dict[num]['PA_Profiles_After_Campaign']+=row.PA_Profiles
                reapp_pa_dict[num]['Reappointment_Revenue_During_Campaign']+=row.Reapp_Rev
                reapp_pa_dict[num]['PA_Profile_Revenue_During_Campaign']+=row.PA_Rev
            else:
                reapp_pa_dict[num]['Reappointments_Before_Campaign']+=row.Reapp_Profiles
                reapp_pa_dict[num]['PA_Profiles_Before_Campaign']+=row.PA_Profiles
        if row.YEAR == 2019:
            reapp_pa_dict[num]['Reappointments_2019']+=row.Reapp_Profiles
            reapp_pa_dict[num]['PA_Profiles_2019']+=row.PA_Profiles
            reapp_pa_dict[num]['Reappointments_Revenue_2019']+=row.Reapp_Rev
            reapp_pa_dict[num]['PA_Profiles_Revenue_2019']+=row.PA_Rev
        elif row.YEAR == 2018:
            reapp_pa_dict[num]['Reappointments_2018']+=row.Reapp_Profiles
            reapp_pa_dict[num]['PA_Profiles_2018']+=row.PA_Profiles
            reapp_pa_dict[num]['Reappointments_Revenue_2018']+=row.Reapp_Rev
            reapp_pa_dict[num]['PA_Profiles_Revenue_2018']+=row.PA_Rev

In [22]:
reapp_pa_df = pd.DataFrame(reapp_pa_dict).transpose().sort_values('Reappointments_After_Campaign', ascending=False)

In [15]:
reapp_start = 0
reapp_more = 0
pa_start = 0
pa_more = 0
reapp_start_rev = 0 
pa_start_rev = 0
reapp_more_rev = 0
pa_more_rev = 0

total_nums = len(reapp_pa_df)
for row in reapp_pa_df.itertuples():
    if row.Reappointments_2018 < row.Reappointments_2019:
        reapp_more +=1
        reapp_net = row.Reappointments_Revenue_2019 - row.Reappointments_Revenue_2018
        reapp_more_rev += reapp_net
    if row.PA_Profiles_2018 < row.PA_Profiles_2019:
        pa_more += 1
        pa_net = row.PA_Profiles_Revenue_2019 - row.PA_Profiles_Revenue_2018
        pa_more_rev += pa_net
    if row.Reappointments_Before_Campaign == 0:
        if row.Reappointments_After_Campaign > 0:
            reapp_start += 1
            reapp_start_rev += row.Reappointments_Revenue_2019
    if row.PA_Profiles_Before_Campaign == 0:
        if row.PA_Profiles_After_Campaign > 0:
            pa_start += 1
            pa_start_rev += row.PA_Profiles_Revenue_2019
print(f'{reapp_start} ({str(reapp_start/total_nums *100)[0:5]}%) customers began using reappointments, generating ${reapp_start_rev} in revenue from reappointments in 2019')
print(f'{pa_start} ({str(pa_start/total_nums *100)[0:5]}%) customers began using PA profiles, generating ${pa_start_rev} in revenue from PA profiles in 2019')
print(f'{reapp_more} ({str(reapp_more/total_nums *100)[0:5]}%) customers purchased more reappointment profiles in 2019 than 2018, generating ${reapp_more_rev} in additional revenue from reappointments from these customers in 2019')
print(f'{pa_more} ({str(pa_more/total_nums *100)[0:5]}%) customers purchased more PA profiles in 2019 than 2018, generating ${pa_more_rev} in additional revenue from PA profiles from these customers in 2019')

9 (0.735%) customers began using reappointments, generating $306 in revenue from reappointments in 2019
11 (0.898%) customers began using PA profiles, generating $408 in revenue from PA profiles in 2019
159 (12.99%) customers purchased more reappointment profiles in 2019 than 2018, generating $18354 in additional revenue from reappointments from these customers in 2019
140 (11.43%) customers purchased more PA profiles in 2019 than 2018, generating $5688 in additional revenue from PA profiles from these customers in 2019


### 2.  Did we see an increase in activity/usage from this population since we began sending marketing communications (started sending communications in Sept-Dec) ?

In [32]:
activity_dict = {}
for num in LH_companies:
    activity_dict[num]={}
    activity_dict[num]['Total_Profiles_Before_Campaign']=0
    activity_dict[num]['Total_Profiles_Jun_to_Aug_2019']=0
    activity_dict[num]['Total_Profiles_Sep_to_Nov_2018']=0
    activity_dict[num]['Total_Profiles_After_Campaign']=0
    activity_dict[num]['Total_Profiles_2019']=0
    activity_dict[num]['Total_Profiles_2018']=0
    
    activity_dict[num]['Total_Revenue_Before_Campaign']=0
    activity_dict[num]['Total_Revenue_Jun_to_Aug_2019']=0
    activity_dict[num]['Total_Revenue_Sep_to_Nov_2018']=0
    activity_dict[num]['Total_Revenue_After_Campaign']=0
    activity_dict[num]['Total_Revenue_2019']=0
    activity_dict[num]['Total_Revenue_2018']=0

In [33]:
for row in all_data.itertuples():
    num = str(row.Company_Name).lower()
    if num in activity_dict.keys():
        if row.YEAR < 2019:
            activity_dict[num]['Total_Profiles_Before_Campaign']+=row.Total_Profiles
            activity_dict[num]['Total_Revenue_Before_Campaign']+=row.Total_Rev
        if row.YEAR == 2019:
            activity_dict[num]['Total_Profiles_2019']+=row.Total_Profiles
            activity_dict[num]['Total_Revenue_2019']+=row.Total_Rev
            if row.MonthName in ['September','October','November','December']:
                activity_dict[num]['Total_Profiles_After_Campaign']+=row.Total_Profiles
                activity_dict[num]['Total_Revenue_After_Campaign']+=row.Total_Rev
            else:
                activity_dict[num]['Total_Profiles_Before_Campaign']+=row.Total_Profiles
                activity_dict[num]['Total_Revenue_Before_Campaign']+=row.Total_Rev
            if row.MonthName in ['June','July','August']:
                activity_dict[num]['Total_Profiles_Jun_to_Aug_2019']+=row.Total_Profiles
                activity_dict[num]['Total_Revenue_Jun_to_Aug_2019']+=row.Total_Rev
        if row.YEAR == 2018:
            activity_dict[num]['Total_Profiles_2018']+=row.Total_Profiles
            activity_dict[num]['Total_Revenue_2018']+=row.Total_Rev
            if row.MonthName in ['September','October','November']:
                activity_dict[num]['Total_Profiles_Sep_to_Nov_2018']+=row.Total_Profiles
                activity_dict[num]['Total_Revenue_Sep_to_Nov_2018']+=row.Total_Rev

In [34]:
activity_df = pd.DataFrame(activity_dict).transpose()

In [40]:
activity_df.sum()

Total_Profiles_Before_Campaign     195478
Total_Profiles_Jun_to_Aug_2019      15053
Total_Profiles_Sep_to_Nov_2018      11524
Total_Profiles_After_Campaign       12893
Total_Profiles_2019                 50682
Total_Profiles_2018                 49480
Total_Revenue_Before_Campaign     4551444
Total_Revenue_Jun_to_Aug_2019      343468
Total_Revenue_Sep_to_Nov_2018      269409
Total_Revenue_After_Campaign       305065
Total_Revenue_2019                1148740
Total_Revenue_2018                1160406
dtype: int64

In [40]:
pro_more_2019 = 0
pro_more_than_before = 0
pro_more_than_last = 0
rev_more_2019 = 0
rev_more_than_before = 0
rev_more_than_last = 0
total_ = len(activity_df)
test_list=[]
for row in activity_df.itertuples():
    if row.Total_Profiles_2018 < row.Total_Profiles_2019:
        pro_more_2019 +=1
    if row.Total_Profiles_Jun_to_Aug_2019 < row.Total_Profiles_After_Campaign:
        pro_more_than_before += 1
    if row.Total_Profiles_Sep_to_Nov_2018 < row.Total_Profiles_After_Campaign:
        pro_more_than_last += 1
        test_list.append(row)
    if row.Total_Revenue_2018 < row.Total_Revenue_2019:
        rev_more_2019 +=1
    if row.Total_Revenue_Jun_to_Aug_2019 < row.Total_Revenue_After_Campaign:
        rev_more_than_before += 1
    if row.Total_Revenue_Sep_to_Nov_2018 < row.Total_Revenue_After_Campaign:
        rev_more_than_last += 1
        
print(f'{pro_more_2019} ({str(pro_more_2019/total_ *100)[0:5]}%) customers bought more profiles in 2019 than 2018.')
print(f'{pro_more_than_before} ({str(pro_more_than_before/total_ *100)[0:5]}%) customers bought more profiles after the campaign (September-November 2019) than the previous three months (June-August 2019).')
print(f'{pro_more_than_last} ({str(pro_more_than_last/total_ *100)[0:5]}%) customers bought more profiles in September-November 2019 than September-November 2018.')

print(f'{rev_more_2019} ({str(rev_more_2019/total_ *100)[0:5]}%) customers generated more revenue in 2019 than 2018.')
print(f'{rev_more_than_before} ({str(rev_more_than_before/total_ *100)[0:5]}%) customers generated more revenue after the campaign (September-November 2019) than the previous three months (June-August 2019).')
print(f'{rev_more_than_last} ({str(rev_more_than_last/total_ *100)[0:5]}%) customers generated more revenue in September-November 2019 than September-November 2018.')

373 (30.47%) customers bought more profiles in 2019 than 2018.
260 (21.24%) customers bought more profiles after the campaign (September-November 2019) than the previous three months (June-August 2019).
363 (29.65%) customers bought more profiles in September-November 2019 than September-November 2018.
332 (27.12%) customers generated more revenue in 2019 than 2018.
246 (20.09%) customers generated more revenue after the campaign (September-November 2019) than the previous three months (June-August 2019).
338 (27.61%) customers generated more revenue in September-November 2019 than September-November 2018.


In [43]:
companies = pd.DataFrame(test_list)

In [44]:
pd.merge(companies, LH_customers, left_on='Index', right_on='Compnay')

,Index,Total_Profiles_Before_Campaign,Total_Profiles_Jun_to_Aug_2019,Total_Profiles_Sep_to_Nov_2018,Total_Profiles_After_Campaign,Total_Profiles_2019,Total_Profiles_2018,Total_Revenue_Before_Campaign,Total_Revenue_Jun_to_Aug_2019,Total_Revenue_Sep_to_Nov_2018,...,Total_Revenue_2019,Total_Revenue_2018,Compnay,First Name,Last Name,Address 1,Address 2,City,State,Zip
0,henry ford health system,4548,373,428,429,1522,1628,10053,0,0,...,0,0,henry ford health system,Lisa,Allen,2799 W Grand Blvd,NaN,Detroit,MI,482022689
1,st lukes university,295,87,48,313,467,141,11803,3480,1920,...,18993,5643,st lukes university,Melissa,Ripper,801 Ostrum St,NaN,Bethlehem,PA,180151000
2,vassar brothers medical center,373,27,20,24,94,84,14473,1080,755,...,3752,3208,vassar brothers medical center,Jackie,Fox,45 Reade Pl,NaN,Poughkeepsie,NY,126013990
3,bassett healthcare,299,31,9,38,127,71,11641,1224,327,...,5050,2696,bassett healthcare,Charlotte,Hoag,1 Atwell Rd,NaN,Cooperstown,NY,133261394
4,arrowhead regional medical center,116,25,6,7,62,36,4540,977,240,...,2444,1397,arrowhead regional medical center,Christine,Fakhreddine,400 N Pepper Ave,NaN,Colton,CA,923241819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,lake granbury medical center,274,40,20,28,100,72,8669,571,563,...,2169,2198,lake granbury medical center,Erin,Jones,1310 Paluxy Rd,NaN,Granbury,TX,760485655
359,kootenai health,1147,119,75,77,359,339,1202,0,0,...,0,0,kootenai health,Lisa,Goodwin,2003 Kootenai Health Way,NaN,Coeur D Alene,ID,838146051
360,sharon hospital,306,18,13,22,66,70,0,0,0,...,0,0,sharon hospital,Pamela,Bartolomeo,50 Hospital Hill Rd,PO Box 789,Sharon,CT,60690789
361,northridge medical center,166,0,2,11,36,23,3659,0,63,...,537,582,northridge medical center,Shelly,Hill,70 Medical Center Dr,NaN,Commerce,GA,305291078


In [48]:
all_data_2 = all_data.drop_duplicates('Company_Name')
all_data_2['Company'] = all_data_2['Company_Name'].str.lower()

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [55]:
pd.merge(companies, all_data_2, left_on='Index', right_on='Company').drop(columns=['Full_Profiles',
'Full_Rev',
'Reapp_Profiles',
'Reapp_Rev',
'PA_Profiles',
'PA_Rev',
'Total_Profiles',
'Total_Rev',
'CM_Rev',
'YEAR',
'MonthName',
'x',
'Company', 'Index']).to_csv('More_Profiles.csv')

### 3.  Did we see an increase in initial profiles used/purchased? 

In [17]:
profiles_dict ={}
for num in LH_companies:
    profiles_dict[num]={}
    profiles_dict[num]['Initial_Profiles_Before_Campaign']=0
    profiles_dict[num]['Initial_Profiles_Jun_to_Aug_2019']=0
    profiles_dict[num]['Initial_Profiles_Sep_to_Nov_2018']=0
    profiles_dict[num]['Initial_Profiles_After_Campaign']=0
    profiles_dict[num]['Initial_Profiles_2019']=0
    profiles_dict[num]['Initial_Profiles_2018']=0

In [18]:
for row in all_data.itertuples():
    num = str(row.Company_Name).lower()
    if num in profiles_dict.keys():
        if row.YEAR < 2019:
            profiles_dict[num]['Initial_Profiles_Before_Campaign']+=row.Full_Profiles
        if row.YEAR == 2019:
            profiles_dict[num]['Initial_Profiles_2019']+=row.Full_Profiles
            if row.MonthName in ['September','October','November','December']:
                profiles_dict[num]['Initial_Profiles_After_Campaign']+=row.Full_Profiles
            else:
                profiles_dict[num]['Initial_Profiles_Before_Campaign']+=row.Full_Profiles
            if row.MonthName in ['June','July','August']:
                profiles_dict[num]['Initial_Profiles_Jun_to_Aug_2019']+=row.Full_Profiles
                
        if row.YEAR == 2018:
            profiles_dict[num]['Initial_Profiles_2018']+=row.Reapp_Profiles
            if row.MonthName in ['September','October','November']:
                profiles_dict[num]['Initial_Profiles_Sep_to_Nov_2018']+=row.Full_Profiles

In [19]:
profiles_df = pd.DataFrame(profiles_dict).transpose()

In [20]:
profiles_df.sum()

Initial_Profiles_Before_Campaign    151116
Initial_Profiles_Jun_to_Aug_2019     11979
Initial_Profiles_Sep_to_Nov_2018      9314
Initial_Profiles_After_Campaign      10212
Initial_Profiles_2019                40531
Initial_Profiles_2018                 8067
dtype: int64

In [27]:
more_2019 = 0
more_than_before = 0
more_than_last = 0
total_ = len(profiles_df)

for row in profiles_df.itertuples():
    if row.Initial_Profiles_2018 < row.Initial_Profiles_2019:
        more_2019 +=1
    if row.Initial_Profiles_Jun_to_Aug_2019 < row.Initial_Profiles_After_Campaign:
        more_than_before += 1
    if row.Initial_Profiles_Sep_to_Nov_2018 < row.Initial_Profiles_After_Campaign:
        more_than_last += 1
        
print(f'{more_2019} ({str(more_2019/total_ *100)[0:5]}%) customers bought more initial profiles in 2019 than 2018.')
print(f'{more_than_before} ({str(more_than_before/total_ *100)[0:5]}%) customers bought more initial profiles after the campaign (September-November 2019) than the previous three months (June-August 2019).')
print(f'{more_than_last} ({str(more_than_last/total_ *100)[0:5]}%) customers bought more initial profiles in September-November 2019 than September-November 2018.')

682 (55.71%) customers bought more initial profiles in 2019 than 2018.
241 (19.68%) customers bought more initial profiles after the campaign (September-November 2019) than the previous three months (June-August 2019).
335 (27.36%) customers bought more initial profiles in September-November 2019 than September-November 2018.


### 4.  From the population that had used less than 3 PA profiles before September 2019, did we see a rise in PA profiles from this population after September 1, 2019? 

In [24]:
reapp_increase = 0
reapp_3 =0
pa_3 = 0
pa_increase = 0
reapp_total = 0
reapp_revenue = 0
pa_total = 0
pa_revenue = 0
total_nums = len(reapp_pa_df)
for row in reapp_pa_df.itertuples():
    if row.Reappointments_Before_Campaign < 3:
        reapp_3 += 1
        if row.Reappointments_After_Campaign > row.Reappointments_Before_Campaign:
            reapp_increase += 1
            reapp_total += row.Reappointments_After_Campaign
            reapp_revenue += row.Reappointment_Revenue_During_Campaign
    if row.PA_Profiles_Before_Campaign <3:
        pa_3 += 1
        if row.PA_Profiles_After_Campaign > row.PA_Profiles_Before_Campaign:
            pa_increase += 1
            pa_total += row.PA_Profiles_After_Campaign
            pa_revenue += row.PA_Profile_Revenue_During_Campaign
print(f'Of the population that used less than 3 PA profiles before September 2019, {pa_increase} customers or {str(pa_increase/pa_3 *100)[0:5]}% had a rise in PA profiles after September 2019')
print(f'These customers purchased {pa_total} profiles during the marketing period, generating ${pa_revenue} in revenue')

Of the population that used less than 3 PA profiles before September 2019, 14 customers or 1.545% had a rise in PA profiles after September 2019
These customers purchased 24 profiles during the marketing period, generating $552 in revenue


### 5.  From the population that had used less than 3 Reappointment profiles before September 2019, did we see a rise in Reappointment profile usage after September 1, 2019?

In [25]:
print(f'Of the population that used less than 3 reappointment profiles before September 2019, {reapp_increase} customers or {str(reapp_increase/reapp_3 *100)[0:5]}% had a rise in reappointment profiles after September 2019')
print(f'These customers purchased {reapp_total} profiles during the marketing period, generating ${reapp_revenue} in revenue')

Of the population that used less than 3 reappointment profiles before September 2019, 11 customers or 1.199% had a rise in reappointment profiles after September 2019
These customers purchased 26 profiles during the marketing period, generating $468 in revenue
